In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [3]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')

In [6]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df['Age'].replace(to_replace = np.nan, value = np.mean(df['Age']),inplace=True)

In [8]:
df['Pclass'].replace(to_replace = [1,2,3], value = ['UpperClass','MiddleClass', 'LowerClass'],inplace=True)

In [9]:
pd.value_counts(df.Pclass)
Pclass = pd.get_dummies(df.Pclass)
df_new = pd.concat([df,Pclass], axis = 'columns')
df_new.drop(['Pclass'], axis='columns', inplace = True)
df_new.head()

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,LowerClass,MiddleClass,UpperClass
0,1,0,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,0,1
2,3,1,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,0,1
4,5,0,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,0,0


In [10]:
df_new.isnull().sum()

PassengerId      0
Survived         0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
LowerClass       0
MiddleClass      0
UpperClass       0
dtype: int64

In [11]:
df1 = df_new[~df_new.Embarked.isnull()].copy()

In [12]:
df1.drop(['Ticket', 'Cabin', 'Name', 'PassengerId'], axis='columns', inplace = True)

In [13]:
df1['Sex'].replace(to_replace = ['male', 'female'], value = [1,0], inplace=True)

In [14]:
pd.value_counts(df1.Embarked)
Embarked = pd.get_dummies(df1.Embarked)
df2 = pd.concat([df1,Embarked], axis = 'columns')
df2.drop(['Embarked'], axis='columns', inplace = True)
df2.head()

,Survived,Sex,Age,SibSp,Parch,Fare,LowerClass,MiddleClass,UpperClass,C,Q,S
0,0,1,22.0,1,0,7.2500,1,0,0,0,0,1
1,1,0,38.0,1,0,71.2833,0,0,1,1,0,0
2,1,0,26.0,0,0,7.9250,1,0,0,0,0,1
3,1,0,35.0,1,0,53.1000,0,0,1,0,0,1
4,0,1,35.0,0,0,8.0500,1,0,0,0,0,1


In [15]:
df2['Fare'] =  df2['Fare'] / np.max(df2['Fare'])

In [16]:
df2.isnull().sum()

Survived       0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
LowerClass     0
MiddleClass    0
UpperClass     0
C              0
Q              0
S              0
dtype: int64

In [17]:
df2.columns

Index(['Survived', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'LowerClass',
       'MiddleClass', 'UpperClass', 'C', 'Q', 'S'],
      dtype='object')

In [18]:
X = df2[['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'LowerClass',
       'MiddleClass', 'UpperClass', 'C', 'Q', 'S']]
X.shape

(889, 11)

In [19]:
y = np.array(df2[['Survived']]).reshape(-1)
y.shape

(889,)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

In [21]:
from keras.models import Sequential
from keras.layers import Dense

In [22]:
# define the keras model
model = Sequential()
model.add(Dense(15, input_dim=11, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [23]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10, verbose=0)

In [25]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

28/28 [==============================] - 0s 1ms/step - loss: 0.3757 - accuracy: 0.8380
Accuracy: 83.80


In [26]:
# make class predictions with the model
from sklearn.metrics import accuracy_score
predictions = model.predict_classes(X_test).reshape(-1)
accuracy = accuracy_score(y_test, predictions, normalize=True, sample_weight=None)
accuracy

NameError: name 'X_test' is not defined

In [ ]:
# from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression(penalty="none", random_state=42, max_iter=10000, solver="saga")
# lr.fit(X_train, y_train)
# train_score = lr.score(X_train, y_train)
# test_score = lr.score(X_test, y_test)
# train_score, test_score

In [ ]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
test

In [ ]:
test.drop(['Cabin','Ticket', 'Name'], axis='columns', inplace = True)
test

In [ ]:
test.isnull().sum()

In [ ]:
test['Age'].replace(to_replace = np.nan, value = np.mean(test['Age']),inplace=True)

In [ ]:
test['Fare'].replace(to_replace = np.nan, value = np.mean(test['Fare']),inplace=True)

In [ ]:
test1 = test.copy()
test1.isnull().sum()

In [ ]:
test1['Sex'].replace(to_replace = ['male', 'female'], value = [1,0],inplace=True)

In [ ]:
test1['Pclass'].replace(to_replace = [1,2,3], value = ['UpperClass','MiddleClass', 'LowerClass'],inplace=True)

In [ ]:
pd.value_counts(test1.Pclass)
Pclass = pd.get_dummies(test1.Pclass)
test3 = pd.concat([test1,Pclass], axis = 'columns')
test3.drop(['Pclass'], axis='columns', inplace = True)
test3.head()

In [ ]:
pd.value_counts(test3.Embarked)
Embarked = pd.get_dummies(test3.Embarked)
test2 = pd.concat([test3,Embarked], axis = 'columns')
test2.drop(['Embarked'], axis='columns', inplace = True)
test2.head()

In [ ]:
test2['Fare'] =  test2['Fare'] / (np.max(test2['Fare']))

In [ ]:
test2.columns

In [ ]:
# X_pred = test2[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
#        'Fare', 'C', 'Q', 'S']]
X_pred = test2[['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'LowerClass',
       'MiddleClass', 'UpperClass', 'C', 'Q', 'S']]
X_pred.shape

In [ ]:
predictions = model.predict_classes(X_pred).reshape(-1)
predictions.shape

In [ ]:
result = pd.DataFrame()
result['Survived'] =  predictions
result['PassengerId'] = test2['PassengerId']
result.set_index("PassengerId", inplace = True)
result

In [ ]:
result.to_csv('/kaggle/working/Predicted_Value.csv')